In [3]:
import sys
import os

openagent_dir = os.path.dirname(os.path.abspath(""))
sys.path.append(openagent_dir)

In [4]:
import re
# from openagent.tools.wrappers.serpapi import SerpAPIWrapper
from openagent import compiler
from openagent.llms._openai import OpenAI

# load .env file in format:
# OPENAI_API_KEY=sk-xxx
from dotenv import load_dotenv
load_dotenv()


llmcc = OpenAI(
    model="gpt-3.5-turbo"
)

llmtc = OpenAI(
    model="text-davinci-003"
)

#### Trying to use directly llm from guidance, without prompt_engine

In [5]:
# # this session, only require a formatted prompt from prompt engine and return a completion
# llmcc_session = llmcc.session()  
# llmcc_session("Hello") # doesn't work


# # it still doesn't work
# llmcc_session('''{{#user~}}
#                 Tell me about Delhi
#                 {{~/user}}
                                            
#                 {{#assistant~}}
#                 {{gen 'info_about_delhi' temperature=0 max_tokens=300}}
#                 {{~/assistant}}''')


# # this is how it should work (prompt should have "<|im_start|>assistant\n" at the end before gen tag)
# # to check the tags like <|im_start|>assistant\n, print output or check prompt.text property
# llmcc_session('''{{#user~}}
#                 Tell me about Delhi
#                 {{~/user}}
                                            
#                 {{#assistant~}}
#                 ''')


# # this works well 
# llmtc_session = llmtc.session()
# llmtc_session(prompt="Hello, my name is Karan, here is my resume - ", caching = False)

In [6]:
# test case to pass only text in assistant tag (to check if it follows the instruction) # works

prompt = compiler('''{{#user~}}
                            Tell me about {{'city'}}
                            {{~/user}}
                                                        
                            {{#assistant~}}
                            Yes, I will tell you about Delhi in Hindi
                            {{~/assistant}}
                       
                            {{#user~}}
                            Yes, tell me
                            {{~/user}}
                                                        
                            {{#assistant~}}
                            {{gen 'info_about_delhi' temperature=0 max_tokens=300}}
                            {{~/assistant}}''', llm = llmcc)

# prompt = prompt()

In [7]:
# test case to pass both text and gen tag (doesn't work because gen tags not supported after text)

prompt = compiler('''{{#user~}}
                            Tell me about Delhi
                            {{~/user}}
                                                        
                            {{#assistant~}}
                            Yes, I will tell you about Delhi in Hindi
                            {{~/assistant}}
                       
                            {{#user~}}
                            Yes, tell me
                            {{~/user}}
                                                        
                            {{#assistant~}}
                            {{gen 'info_about_delhi' temperature=0 max_tokens=300}}
                       
                            also here is the info about nearby cities 
                       
                            {{gen 'info_about_delhi_nearby_cities' temperature=0 max_tokens=300}}
                            {{~/assistant}}''', llm = llmcc)

prompt = prompt()

Stop program {{#user~}} 
 Tell me about Delhi
 {{~/user}} 
 
 {{#assistant~}} 
 Yes, I will tell you about Delhi in Hindi
 {{~/assistant}} 
 
 {{#user~}} 
 Yes, tell me
 {{~/user}} 
 
 {{#assistant~}} 
 {{gen 'info_about_delhi' temperature=0 max_tokens=300}} 
 
 also here is the info about nearby cities 
 
 {{gen 'info_about_delhi_nearby_cities' temperature=0 max_tokens=300}} 
 {{~/assistant}}

In [ ]:
# test case to pass both text and gen tag in assistant (properly!) to guide llm for further outputs (works). Only text after gen tag works, not before

prompt = compiler('''{{#user~}}
                            Tell me about Delhi
                            {{~/user}}
                                                        
                            {{#assistant~}}
                            Yes, I will tell you about Delhi in Hindi
                            {{~/assistant}}
                       
                            {{#user~}}
                            Yes, tell me
                            {{~/user}}
                                                        
                            {{#assistant~}}
                            {{gen 'info_about_delhi' temperature=0 max_tokens=300}}
                            Let me also tell about some nearby cities to Delhi:
                            {{~/assistant}}

                            {{#assistant~}}
                             {{gen 'info_about_nearby_cities' temperature=0 max_tokens=300}}
                              let me also tell you about Jaipur
                            {{~/assistant}}
                       
                              {{#assistant~}}
                             {{gen 'info_about_jaipur' temperature=0 max_tokens=300}}
                              
                            {{~/assistant}}
                       
                       ''', llm = llmcc)

prompt = prompt()

In [ ]:
# test case to check output formatting using text completion llm

prompt = compiler(
'''Here is something you should know about {{actor}}
He was a short man, may be {{gen 'height' max_tokens=2 top_p =1.0}} height, and had {{gen 'hair_color' max_tokens=2 top_p = 1.0}} hair.
He was known for his {{gen 'quality1' max_tokens= 1}} and {{gen 'quality2' max_tokens=1}}
''', llm = llmtc, actor = "")

# print(str(prompt))

prompt.variables()

# prompt._variables
# prompt = prompt(actor = "Tyrion Lannister")
# prompt = prompt()

In [ ]:
# test case to check dynamic flow tests: works

dynamic_flow = compiler(
                        """{{#user~}}
                            Tell me about Delhi. Cover history, famous places, food, culture, and majorly IT industry.
                            {{~/user}}
                                                        
                            {{#assistant~}}
                            {{gen 'info_about_delhi' temperature=0 max_tokens=300}}
                            {{~/assistant}}

                            {{#user~}}
                            Tell me about Bengaluru. Cover history, famous places, food, culture, and majorly IT industry.
                            {{~/user}}
                                                        
                            {{#assistant~}}
                            {{gen 'info_about_bengaluru' temperature=1 max_tokens=200}}
                            {{~/assistant}}
                                                        
                            {{#user~}}
                            Consider {{info_about_delhi}} AND {{info_about_bengaluru}} and tell me which city is better than other in terms of Startups and Founders
                            {{~/user}}
                                                        
                            {{#assistant~}}
                            {{gen 'response' temperature=1 max_tokens=400}}
                            {{~/assistant}}

                            {{#user~}}
                            Which city has more population?
                            {{~/user}}
                                                        
                            {{#assistant~}}
                            {{gen 'response' temperature=1 max_tokens=400}}
                            {{~/assistant}}                                                                 
                    """
                    )

prompt = dynamic_flow()
# working

In [ ]:
# test case to check chat loop: works

chat_loop1 = compiler(
                '''{{#system}}You are a helpful assistant{{/system}}

                    {{~#geneach 'conversation' stop=False}}

                    {{#user~}}
                    {{set 'this.user_text' (await 'user_text') hidden=False}}
                    {{~/user}}

                    {{#assistant~}}
                    {{gen 'this.response' temperature=0 max_tokens=300}}
                    {{~/assistant}}
                    
                    {{~/geneach}}''') 


# manual conversation (in manual conversation, next input must be provide to return value)
chat_loop2 = chat_loop1(user_text = "My name is yash") # chat_loop2 has the conversation of chat_loop1
chat_loop3 = chat_loop2(user_text = "What is my name?") # chat_loop3 has the conversation of chat_loop2


# chat loop (in a loop, both function (chat_loop1) and returned value (chat_loop2) should have same name)
n = 0
while n < 5:
    chat_loop = chat_loop(user_text = input())
    print(chat_loop['conversation'])
    n += 1


In [ ]:
# chatbot demonstration

def chatbot(input, chat_loop):

    if chat_loop is None:
        chat_loop = compiler(
                    '''{{#system}}You are a helpful assistant{{/system}}

                        {{~#geneach 'conversation' stop=False}}

                        {{#user~}}
                        {{set 'this.user_text' (await 'user_text') hidden=False}}
                        {{~/user}}

                        {{#assistant~}}
                        {{gen 'this.response' temperature=0 max_tokens=300}}
                        {{~/assistant}}
                        
                        {{~/geneach}}''', llm = llmcc, silent = True) 
    
    chat_loop = chat_loop(user_text = input) 
    return chat_loop['conversation'][-2]['response'], chat_loop

In [ ]:
resp, loop = chatbot("Hello", None)
print(resp)

In [ ]:
resp, loop = chatbot("How do we train a ML model? Explain in 50 words only", loop)
print(resp)